In [2]:
import os
import json
import ee
import geemap
from geemap import geojson_to_ee



cloud_project = 'hotspotstoplight'

ee.Initialize(project = cloud_project)


# load aoi
file_path = os.path.join('../../inputs/san_jose_aoi/resourceshedbb_CostaRica_SanJose.geojson')
absolute_path = os.path.abspath(file_path)

with open(absolute_path) as f:
    json_data = json.load(f)

aoi = geojson_to_ee(json_data) # need as a feature collection, not bounding box

## set time frame
before_start= '2017-09-25'
before_end='2017-10-05'

after_start='2017-10-05'
after_end='2017-10-15'

## get SAR data
print("Getting SAR data")

# SET SAR PARAMETERS (can be left default)

# Polarization (choose either "VH" or "VV")
polarization = "VH"  # or "VV"

# Pass direction (choose either "DESCENDING" or "ASCENDING")
pass_direction = "DESCENDING"  # or "ASCENDING"

# Difference threshold to be applied on the difference image (after flood - before flood)
# It has been chosen by trial and error. Adjust as needed.
difference_threshold = 1.25

# Relative orbit (optional, if you know the relative orbit for your study area)
# relative_orbit = 79

# Load and filter Sentinel-1 GRD data by predefined parameters
collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
    .filter(ee.Filter.eq('orbitProperties_pass', pass_direction)) \
    .filter(ee.Filter.eq('resolution_meters', 10)) \
    .filterBounds(aoi) \
    .select(polarization)

# Select images by predefined dates
before_collection = collection.filterDate(before_start, before_end)
after_collection = collection.filterDate(after_start, after_end)

# Create a mosaic of selected tiles and clip to the study area
before = before_collection.mosaic().clip(aoi)
after = after_collection.mosaic().clip(aoi)

# Apply radar speckle reduction by smoothing
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters')
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters')

# Calculate the difference between the before and after images
difference = after_filtered.divide(before_filtered)

# Apply the predefined difference-threshold and create the flood extent mask
threshold = difference_threshold
difference_binary = difference.gt(threshold)

# Refine the flood result using additional datasets
swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality')
swater_mask = swater.gte(10).updateMask(swater.gte(10))
flooded_mask = difference_binary.where(swater_mask, 0)
flooded = flooded_mask.updateMask(flooded_mask)
connections = flooded.connectedPixelCount()
flooded = flooded.updateMask(connections.gte(8))

# Mask out areas with more than 5 percent slope using a Digital Elevation Model
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM')
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
flooded = flooded.updateMask(slope.lt(5))

## write out to file
print("Writing out to file")
crs = flooded.projection().crs()

flooded = flooded.setDefaultProjection(crs=crs, scale=10)

Getting SAR data
Writing out to file


In [3]:

# load stream distance for crs
stream_dist_proximity_collection = ee.ImageCollection("projects/sat-io/open-datasets/HYDROGRAPHY90/stream-outlet-distance/stream_dist_proximity")\
    .filterBounds(aoi)\
    .mosaic()
stream_dist_proximity = stream_dist_proximity_collection.clip(aoi).rename('stream_distance')
hydro_proj = stream_dist_proximity.projection()

hydro_proj = stream_dist_proximity.projection()

dem = ee.Image('USGS/SRTMGL1_003').clip(aoi)

# Set the default projection from the hydrography dataset
flooded = flooded.setDefaultProjection(hydro_proj)

# Now, reduce the resolution
flooded_mode = flooded.reduceResolution(
    reducer=ee.Reducer.mode(),
    maxPixels=10000
).reproject(
    crs=hydro_proj
)

# Reproject the flooded image to match the DEM's projection
dem_projection = dem.projection()
flooded_reprojected = flooded.reproject(crs=dem_projection)


san_jose = ee.Geometry.Polygon([
    [
        [-84.52073469866146, 9.554427321389259],
        [-83.61258439158091, 9.554427321389259],
        [-83.61258439158091, 10.280374182928231],
        [-84.52073469866146, 10.280374182928231],
        [-84.52073469866146, 9.554427321389259]
    ]
])

filename = '../../outputs/test.tif'

geemap.ee_export_image(flooded_reprojected, filename, scale=100, crs=dem_projection, region=san_jose)

Generating URL ...
Please wait ...
Data downloaded to c:\Users\Nissim\Documents\GitHub\Climate\flood_mapping\data\outputs\test.tif
